In [1]:
import pandas as pd
import re
import emoji
import nltk
import json
import numpy as np
import csv

In [10]:
SMM18 = pd.read_csv("SMM4H18_Train.csv", sep='\t')
SMM18.head()

,tweet_id,user_id,text,created_at,label,Drug Name Extracted,Drug Name Normalized
0,46993489828909057,182266659,@AirLewis VITAMIN C AND GARLIC CRUSHING MEOW,2011-03-13 17:58:15,0,NaN,NaN
1,397783574797352960,545313028,Only 3 Arnica Balms left at Clinic Alternative...,2013-11-05 17:52:49,1,arnica balm,arnica balm
2,400641718372270080,388247063,Recent research from Holland suggests folic ac...,2013-11-13 15:10:03,0,NaN,NaN
3,401583766973734912,1590139340,I'm so burnt and this aloe isn't helping 😡,2013-11-16 05:33:25,0,NaN,NaN
4,404288692514078720,34971386,@kristalcraven sudafed that I'm not sure I'm c...,2013-11-23 16:41:50,1,sudafed,sudafed


In [11]:
#import all three tsv file
tt3_1 = pd.read_csv('BioCreative_TrainTask3.1.tsv', sep='\t')
tt3_0 = pd.read_csv('BioCreative_TrainTask3.0.tsv', sep='\t')
tt3_ = pd.read_csv('BioCreative_ValTask3.tsv', sep='\t')


#DSV file pre-processing 

In [19]:
tt3_1.head()

,tweet_id,user_id,created_at,text,start,end,span,drug
0,637431359485673473,15954214,2015-08-29,@kellyrelish I preordered this! http://t.co/HT...,-,-,-,-
1,781144946866737153,1228869366,2016-09-28,Like how do you even think it's okay to do tha...,-,-,-,-
2,786928869949386752,132441180,2016-10-14,"Well done, @IUBloomington. See you in November...",-,-,-,-
3,464414606409359360,436230026,2014-05-08,Our lights went out and I was in the bathroom ...,-,-,-,-
4,494331995246166017,511257055,2014-07-30,I had my first back to school nightmare last n...,-,-,-,-


In [13]:
#emoji range "book"
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

In [14]:
def trimmer(tweet):
    tweet = re.sub("@[A-Za-z0-9.,!?]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = emoji_pattern.sub(r'', tweet) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    return tweet

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/shan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
#using default tokenizer get the clean text and its token counts
from nltk.tokenize import word_tokenize


tt3_1["trimmed text"] = [trimmer(tweet) for tweet in tt3_1["text"]]
tt3_1["nltk tokenized length"] = [len(word_tokenize(tweet)) for tweet in tt3_1["trimmed text"]]

tt3_0["trimmed text"] = [trimmer(tweet) for tweet in tt3_0["text"]]
tt3_0["nltk tokenized length"] = [len(word_tokenize(tweet)) for tweet in tt3_0["trimmed text"]]

tt3_["trimmed text"] = [trimmer(tweet) for tweet in tt3_["text"]]
tt3_["nltk tokenized length"] = [len(word_tokenize(tweet)) for tweet in tt3_["trimmed text"]]


In [18]:
SMM18["trimmed text"] = [trimmer(tweet) for tweet in SMM18["text"]]
SMM18["nltk tokenized length"] = [len(word_tokenize(tweet)) for tweet in SMM18["trimmed text"]]

In [19]:
#adding Bert tokenizer:

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tt3_1["BERT tokenized length"] = [len(tokenizer.tokenize(tweet)) for tweet in tt3_1["trimmed text"]]

tt3_0["BERT tokenized length"] = [len(tokenizer.tokenize(tweet)) for tweet in tt3_0["trimmed text"]]

tt3_["BERT tokenized length"] = [len(tokenizer.tokenize(tweet)) for tweet in tt3_["trimmed text"]]

SMM18["BERT tokenized length"] = [len(tokenizer.tokenize(tweet)) for tweet in SMM18["trimmed text"]]

In [20]:
tt3_.head()

,tweet_id,user_id,created_at,text,start,end,span,drug,trimmed text,nltk tokenized length,BERT tokenized length
0,675404777556176896,1152401473,2015-12-11,I'm 37 weeks pregnant so I can do whatever the...,-,-,-,-,I'm 37 weeks pregnant so I can do whatever the...,14,15
1,217111669657178112,24324898,2012-06-25,@RaynaJaye bahahahahahahahaha,-,-,-,-,bahahahahahahahaha,1,8
2,553397745701376000,2333890110,2015-01-09,@Mamas_RipDad thank yu 😘,-,-,-,-,RipDad thank yu,3,4
3,822787020183441409,1486889246,2017-01-21,Bruh do you wanna fight nah man it's problem,-,-,-,-,Bruh do you wanna fight nah man it's problem,11,12
4,580425692548108289,1435663525,2015-03-24,It has been a difficult transition .,-,-,-,-,It has been a difficult transition .,7,7


In [21]:
tt3_0.head()

,tweet_id,user_id,created_at,text,start,end,span,drug,trimmed text,nltk tokenized length,BERT tokenized length
0,525872716411580416,2333890110,2014-10-25,@Rhy_QD10 yeah irking he need his ass whoop I ...,-,-,-,-,QD10 yeah irking he need his ass whoop I real...,24,30
1,809577207597244417,165916824,2016-12-16,Panda Express 🐼😛,-,-,-,-,Panda Express,2,2
2,590918768269864960,2414667758,2015-04-22,Well..technology wins agains. People are fight...,-,-,-,-,Well..technology wins agains. People are fight...,22,26
3,237385144221192193,24324898,2012-08-20,@jennabennabear what happened?????,-,-,-,-,what happened?????,7,7
4,166288274300735488,181819579,2012-02-05,A first grade teacher asked her class to compl...,-,-,-,-,A first grade teacher asked her class to compl...,22,24


In [22]:
tt3_1.head()

,tweet_id,user_id,created_at,text,start,end,span,drug,trimmed text,nltk tokenized length,BERT tokenized length
0,637431359485673473,15954214,2015-08-29,@kellyrelish I preordered this! http://t.co/HT...,-,-,-,-,I preordered this!,4,6
1,781144946866737153,1228869366,2016-09-28,Like how do you even think it's okay to do tha...,-,-,-,-,Like how do you even think it's okay to do tha...,32,35
2,786928869949386752,132441180,2016-10-14,"Well done, @IUBloomington. See you in November...",-,-,-,-,"Well done, See you in November! Indiana IU Hoo...",13,17
3,464414606409359360,436230026,2014-05-08,Our lights went out and I was in the bathroom ...,-,-,-,-,Our lights went out and I was in the bathroom ...,16,17
4,494331995246166017,511257055,2014-07-30,I had my first back to school nightmare last n...,-,-,-,-,I had my first back to school nightmare last n...,11,11


In [23]:
SMM18.head()

,tweet_id,user_id,text,created_at,label,Drug Name Extracted,Drug Name Normalized,trimmed text,nltk tokenized length,BERT tokenized length
0,46993489828909057,182266659,@AirLewis VITAMIN C AND GARLIC CRUSHING MEOW,2011-03-13 17:58:15,0,NaN,NaN,VITAMIN C AND GARLIC CRUSHING MEOW,6,7
1,397783574797352960,545313028,Only 3 Arnica Balms left at Clinic Alternative...,2013-11-05 17:52:49,1,arnica balm,arnica balm,Only 3 Arnica Balms left at Clinic Alternative...,23,28
2,400641718372270080,388247063,Recent research from Holland suggests folic ac...,2013-11-13 15:10:03,0,NaN,NaN,Recent research from Holland suggests folic ac...,19,22
3,401583766973734912,1590139340,I'm so burnt and this aloe isn't helping 😡,2013-11-16 05:33:25,0,NaN,NaN,I'm so burnt and this aloe isn't helping,10,13
4,404288692514078720,34971386,@kristalcraven sudafed that I'm not sure I'm c...,2013-11-23 16:41:50,1,sudafed,sudafed,sudafed that I'm not sure I'm comfortable taki...,23,28


In [69]:
#Output to CSVs
tt3_1.to_csv('cleaned_TrainTask3.1.csv', encoding='utf-8')
tt3_0.to_csv('cleaned_TrainTask3.0.csv', encoding='utf-8')
tt3_.to_csv('cleaned_TrainTask3.csv', encoding='utf-8')

In [24]:
SMM18.to_csv("cleaned_SMM4H18_Train.csv", encoding='utf-8')